In [2]:
import pandas as pd
import sagemaker
from sagemaker import get_execution_role
import json
import boto3


In [112]:
 fully_engineered = pd.read_csv('s3://zillow-kaggle-awsml-2020/fully_engineered.csv', index_col=0)

In [113]:
for i, n in enumerate(fully_engineered.columns.values):
    print ( str(i) +'\t' + n)

0	parcelid
1	logerror
2	transactiondate
3	bathroom_overage
4	knockdown
5	unreliable_assessment
6	low_qual_building
7	med_qual_building
8	assessed_structure_cost_per_sqft_outlier
9	topmarket
10	bedroom_overage
11	lotsizelevel
12	lowstructuretaxoutlier
13	lowlandtaxoutlier
14	multifam
15	highdensity
16	typical_land_use
17	single_story
18	has_pool
19	istrunc_calculatedfinishedsquarefeet
20	trunc_calculatedfinishedsquarefeet
21	istrunc_calculatedbathnbr
22	trunc_calculatedbathnbr
23	no_garage_info
24	garage_size_level
25	fireplace_level
26	patio_level
27	has_large_accessory
28	log_age
29	basement_level
30	full_basement
31	square_foot_overage_small
32	square_foot_overage_large
33	square_foot_overage_opulent
34	recent_tax_delinquency
35	is_trunc_bedrooms
36	trunc_bedrooms
37	is_trunc_bathrooms
38	trunc_bathrooms
39	lot_size_overage_small
40	lot_size_overage_mid
41	lot_size_overage_large
42	lot_size_overage_huge
43	lot_size_overage_raw
44	has_unreliable_bedrooms
45	has_unreliable_bathrooms
46

In [115]:
valid_features = fully_engineered.columns.values[3:96].tolist()

In [64]:
valid_features

['bathroom_overage',
 'knockdown',
 'unreliable_assessment',
 'low_qual_building',
 'med_qual_building',
 'assessed_structure_cost_per_sqft_outlier',
 'topmarket',
 'bedroom_overage',
 'lotsizelevel',
 'lowstructuretaxoutlier',
 'lowlandtaxoutlier',
 'multifam',
 'highdensity',
 'typical_land_use',
 'single_story',
 'has_pool',
 'istrunc_calculatedfinishedsquarefeet',
 'trunc_calculatedfinishedsquarefeet',
 'istrunc_calculatedbathnbr',
 'trunc_calculatedbathnbr',
 'no_garage_info',
 'garage_size_level',
 'fireplace_level',
 'patio_level',
 'has_large_accessory',
 'log_age',
 'basement_level',
 'full_basement',
 'square_foot_overage_small',
 'square_foot_overage_large',
 'square_foot_overage_opulent',
 'recent_tax_delinquency',
 'is_trunc_bedrooms',
 'trunc_bedrooms',
 'is_trunc_bathrooms',
 'trunc_bathrooms',
 'lot_size_overage_small',
 'lot_size_overage_mid',
 'lot_size_overage_large',
 'lot_size_overage_huge',
 'lot_size_overage_raw',
 'has_unreliable_bedrooms',
 'has_unreliable_bath

In [116]:
holdout_split = fully_engineered.sample(frac=.7, random_state=33)
engineered_trn = fully_engineered.drop(holdout_split.index)

In [117]:
engineered_tst = holdout_split.sample(frac=.4, random_state=37)
engineered_val = holdout_split.drop(engineered_tst.index)

In [120]:
experiments_names_and_targets = {'low_binary':'neg_loss', 'high_binary':'pos_loss', 'regression':'logerror'}
for name,label_col in experiments_names_and_targets.items():
    temp = engineered_trn[[label_col] + valid_features]
    #temp.reset_index()
    temp = 1.0*temp
    temp.to_csv('train1.csv', index=False, header=False)
    !aws s3 cp train1.csv {'s3://zillow-kaggle-awsml-2020/{}/'.format(name)}
    temp = engineered_val[[label_col] + valid_features]
    #temp.reset_index()
    temp = 1.0*temp
    temp.to_csv('valid1.csv', index=False, header=False)
    !aws s3 cp valid1.csv {'s3://zillow-kaggle-awsml-2020/{}/'.format(name)}
    temp = engineered_tst[[label_col] + valid_features]
    #temp.reset_index()
    temp = 1.0*temp
    temp.to_csv('test1.csv', index=False, header=False)
    !aws s3 cp test1.csv {'s3://zillow-kaggle-awsml-2020/{}/'.format(name)}

upload: ./train1.csv to s3://zillow-kaggle-awsml-2020/low_binary/train1.csv
upload: ./valid1.csv to s3://zillow-kaggle-awsml-2020/low_binary/valid1.csv
upload: ./test1.csv to s3://zillow-kaggle-awsml-2020/low_binary/test1.csv
upload: ./train1.csv to s3://zillow-kaggle-awsml-2020/high_binary/train1.csv
upload: ./valid1.csv to s3://zillow-kaggle-awsml-2020/high_binary/valid1.csv
upload: ./test1.csv to s3://zillow-kaggle-awsml-2020/high_binary/test1.csv
upload: ./train1.csv to s3://zillow-kaggle-awsml-2020/regression/train1.csv
upload: ./valid1.csv to s3://zillow-kaggle-awsml-2020/regression/valid1.csv
upload: ./test1.csv to s3://zillow-kaggle-awsml-2020/regression/test1.csv


In [59]:
engineered_trn.iloc[10, 17:25]

single_story                            False
has_pool                                False
istrunc_calculatedfinishedsquarefeet    False
trunc_calculatedfinishedsquarefeet        NaN
istrunc_calculatedbathnbr               False
trunc_calculatedbathnbr                   NaN
no_garage_info                          False
garage_size_level                           0
Name: 126, dtype: object

In [24]:
feats

array(['hellobathroom_overage', 'helloknockdown',
       'hellounreliable_assessment', 'hellolow_qual_building',
       'hellomed_qual_building',
       'helloassessed_structure_cost_per_sqft_outlier', 'hellotopmarket',
       'hellobedroom_overage', 'hellolotsizelevel',
       'hellolowstructuretaxoutlier', 'hellolowlandtaxoutlier',
       'hellomultifam', 'hellohighdensity', 'hellotypical_land_use',
       'hellosingle_story', 'hellohas_pool',
       'helloistrunc_calculatedfinishedsquarefeet',
       'hellotrunc_calculatedfinishedsquarefeet',
       'helloistrunc_calculatedbathnbr', 'hellotrunc_calculatedbathnbr',
       'hellono_garage_info', 'hellogarage_size_level',
       'hellofireplace_level', 'hellopatio_level',
       'hellohas_large_accessory', 'hellolog_age', 'hellobasement_level',
       'hellofull_basement', 'hellosquare_foot_overage_small',
       'hellosquare_foot_overage_large',
       'hellosquare_foot_overage_opulent', 'hellorecent_tax_delinquency',
       'hellois_

In [3]:
bucket = 'zillow-kaggle-awsml-2020'
prefix = 'engineered'

In [ ]:
region_name = boto3.Session().region_name
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'linear-learner')


In [75]:
nan_cols = fully_engineered.isna().any()

In [76]:
nan_cols[nan_cols==True]

Series([], dtype: bool)

In [87]:
fully_engineered.iloc[:,48]

0         8.926518
1         8.200837
2         9.343472
3        11.168461
4         8.699681
5         7.904335
6         8.806574
7         0.000000
8         0.000000
9         0.000000
10        9.021598
11        8.861208
12        9.013717
13        0.000000
14        8.939450
15        8.699681
16        8.641709
17       11.901394
18        8.881975
19        8.727778
20        8.399535
21        8.360773
22        8.699681
23        9.298992
24        7.418181
25        9.017726
26        8.806424
27        0.000000
28       10.682812
29        8.993676
           ...    
77583     8.412055
77584     9.006999
77585     8.922125
77586     8.227910
77587     8.671630
77588     8.919587
77589     8.651374
77590    10.580505
77591     9.791550
77592     0.000000
77593    10.021137
77594     8.985320
77595     8.723231
77596     8.636397
77597     8.556798
77598     9.392745
77599     8.401782
77600    10.723245
77601    11.584250
77602     8.516393
77603     8.900276
77604     8.